# Import Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx
from tqdm import tqdm
from copy import deepcopy
import pandas as pd
from desdeo_mcdm.interactive.NautilusNavigator import NautilusNavigator

In [2]:
from graph import Graph, Point
from optimization import generate_rand_points, crossover, dominates, non_dominated_sort, crowding_distance, random_connected_partitions

# Generate Graph

In [ ]:
np.random.seed(1)

n_points = 45
n_partition = 3
interval = [-30, 30]

graph = generate_rand_points(Graph(), n_points, interval)
points_x, points_y = graph.get_points_coords()

weight = graph.calculate_weight(1, 2)
pos = {i: (graph.get_points()[i].get("x"), graph.get_points()[i].get("y")) for i in graph.get_points()}

G = nx.Graph(np.array(weight))
nx.draw(G, node_size=50, pos=pos)
plt.axis("on")
plt.show()

In [ ]:
graph

# NSGA 2

## Objective Function

In [ ]:
def f1_obj(edge_cut):
    sum = 0
    for i in edge_cut: 
        sum += edge_cut.get(i, 0)
    return sum

def f2_obj(graph, n):
    sum = 0
    for i in range(n - 1):
        for j in range(i, n):
            sum += np.abs(graph.get_partition_size(i + 1) - graph.get_partition_size(j + 1)) 
    return sum

def f3_obj(graph, n):
    sum = 0
    for i in range(n):
        sum += graph.get_partition_spread(i + 1)
    return sum

In [ ]:
mutation_rate = 0.8
crossover_rate = 0.3
n_iteration = 500
n_population = 500

In [ ]:
populations = np.array([])
for i in tqdm(range(n_population)):
    new_graph = deepcopy(graph) 
    random_connected_partitions(new_graph, weight, n_partition)
    populations = np.append(new_graph, populations)

for i in tqdm(range(n_iteration)):
    populations_with_offspring = np.copy(populations)

    for i in range(n_population // 2):
        a = deepcopy(populations[np.random.randint(0, n_population)])
        b = deepcopy(populations[np.random.randint(0, n_population)])
        
        populations = np.append(new_graph, populations)
        if(np.random.rand() <= crossover_rate):
            a, b = crossover(a, b, n_partition)
        if(np.random.rand() <= mutation_rate):
            a.mutate_graph()
        if(np.random.rand() <= mutation_rate):
            b.mutate_graph()
        
        populations_with_offspring = np.append(populations_with_offspring, [a, b])

    f_list = np.array([])
    for i, population in enumerate(populations_with_offspring):
        f_list = np.append(f_list, [i, f1_obj(population.get_edge_cut()), f2_obj(population, n_partition), f3_obj(population, n_partition)])

    f_list = f_list.reshape(n_population * 2, 4)

    fronts = non_dominated_sort(f_list)
    fronts = fronts[:len(fronts) - 1]
    crowding_distance_list = []

    for i, front in enumerate(fronts):
        crowding_distance_list.append([])
        crowding_distance_list[i] = crowding_distance(front, f_list[:, 1:])

    new_population = np.array([])

    for i, front in enumerate(fronts):
        front_temp = sorted([[front[j], crowding_distance_list[i][j]] for j in range(len(crowding_distance_list[i]))], key=lambda x: x[1], reverse=True)
        for val in front_temp:
            if(len(new_population) < n_population):
                new_population = np.append(new_population, populations_with_offspring[val[0]])
            else: 
                break
        else:
            continue
        break

    populations = np.copy(new_population)

In [ ]:
f_list = np.array([])

for i, population in enumerate(populations):
    f_list = np.append(f_list, [f1_obj(population.get_edge_cut()), f2_obj(population, n_partition), f3_obj(population, n_partition)])

f_list = f_list.reshape(n_population, 3)

px.scatter_3d(x=f_list[:, 0], y=f_list[:, 1], z=f_list[:, 2], opacity=0.5)

## Generate CSV

In [ ]:
columns = np.append(np.array(['f1', 'f2', 'f3']), ['var' for i in range(n_points)])
output = pd.DataFrame(columns=columns)

for population in populations: 
    f1, f2, f3 = f1_obj(population.get_edge_cut()), f2_obj(population, n_partition), f3_obj(population, n_partition)
    var = population.get_partition()
    data = np.append([f1, f2, f3], [val for val in var])
    output.loc[len(output)] = data

output.to_excel('output.xlsx')

## Load CSV

In [56]:
output = pd.read_excel('output.xlsx')
f1, f2, f3 = output.f1, output.f2, output.f3

In [57]:
px.scatter_3d(output, x='f1', y='f2', z='f3', opacity=0.5)

# Interactive

In [58]:
front = np.array([f1, f2, f3], dtype=object).T
nadir = np.array([np.max(f1), np.max(f2), np.max(f3)])
ideal = np.array([np.min(f1), np.min(f2), np.min(f3)])
method = NautilusNavigator(front, ideal, nadir)

print(f"Nadir Point: {nadir}")
print(f"Ideal Point: {ideal}")

Nadir Point: [26.40028127 68.         71.59929186]
Ideal Point: [ 5.33771261  8.         33.50317983]


In [59]:
req_first = method.start()

print(req_first)
print(req_first.content.keys())

dict_keys(['message', 'ideal', 'nadir', 'reachable_lb', 'reachable_ub', 'user_bounds', 'reachable_idx', 'step_number', 'steps_remaining', 'distance', 'allowed_speeds', 'current_speed', 'navigation_point'])


In [60]:
print(req_first.content["message"])

Please supply aspirations levels for each objective between the upper and lower bounds as `reference_point`. Specify a speed between 1-5 as `speed`. If going to a previous step is desired, please set `go_to_previous` to True, otherwise it should be False. Bounds for one or more objectives may also be specified as 'user_bounds'; when navigating,the value of the objectives present in the navigation points will not exceed the valuesspecified in 'user_bounds'.Lastly, if stopping is desired, `stop` should be True, otherwise it should be set to False.


In [61]:
print(req_first.content["reachable_lb"])
print(req_first.content["reachable_ub"])

[ 5.33771261  8.         33.50317983]
[26.40028127 68.         71.59929186]


In [51]:
reference_point = np.array([15, 30, 50])
go_to_previous = False
stop = False
speed = 1

response = dict(reference_point=reference_point, go_to_previous=False, stop=False, speed=1, user_bounds=[None, None])

In [52]:
req_first.response = response
req_snd = method.iterate(req_first)

print(req_snd.content["reachable_lb"])
print(req_snd.content["reachable_ub"])

NautilusNavigatorException: An exception rose when validating the given user bounds [nan nan].
Previous exception: <class 'desdeo_mcdm.interactive.NautilusNavigator.NautilusNavigatorException'>: The given user bounds '[nan nan]' has mismatching dimensions compared to the ideal point '[ 5.33771261  8.         33.50317983]'..